# Compass.UOL - Sprint 4 & 5
## Equipe 3 - José Pedro, Pedro Montenegro, Natália Cardoso, Renan Mazzilli


## Dataset: [hotel_reservations](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset)

## Configuração de infraestrutura necessária e roles de acesso no AWS

### Function de criação da infraestrutura no DynamoDB, S3 Bucket e IAM Role Access Permissions para o SageMaker

### Imports e instalações necessárias

In [1]:
# Instalar as bibliotecas necessárias para o projeto
%pip install pandas boto3 joblib scikit-learn sagemaker seaborn

# Importar as bibliotecas necessárias 
import json
import boto3
from botocore.exceptions import ClientError

Note: you may need to restart the kernel to use updated packages.


### Configurações gerais do projeto e variáveis globais

In [2]:
# Carregar configurações do arquivo JSON
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Atribuir as variáveis de configuração
bucket_name = config['bucket_name']
table_name = config['table_name']
model_metrics_table = config['model_metrics_table']
role_name = config['role_name']
dataset_path = config['dataset_path']
processed_dataset_path = config['processed_dataset_path']
model_path = config['model_path']
profile_name = config['profile_name']
aws_region = config['aws_region']

# Inicialização da sessão AWS
session = boto3.Session(profile_name=profile_name, region_name=aws_region)
s3_client = session.client('s3')
dynamodb_client = session.client('dynamodb')
iam_client = session.client('iam')

In [3]:
# Função para criar a tabela DynamoDB
class AWSResourceManager:
    # Inicialização da classe com o nome do perfil
    def __init__(self, profile_name):
        self.profile_name = profile_name
        self.session = boto3.Session(profile_name=self.profile_name, region_name='us-east-1')
        self.dynamodb_client = self.session.client('dynamodb')
        self.s3_client = self.session.client('s3')
        self.iam_client = self.session.client('iam')
    # Função para criar a tabela DynamoDB com o nome da tabela
    def create_dynamodb_table(self, table_name):
        try:
            # Criação da tabela com a chave primária Booking_ID
            self.dynamodb_client.create_table(
                TableName=table_name,
                KeySchema=[
                    {
                        'AttributeName': 'Booking_ID',
                        'KeyType': 'HASH'
                    }
                ],
                AttributeDefinitions=[
                    {
                        'AttributeName': 'Booking_ID',
                        'AttributeType': 'S'
                    }
                ],
                ProvisionedThroughput={
                    'ReadCapacityUnits': 5,
                    'WriteCapacityUnits': 5
                }
            )
            print(f'Table {table_name} created successfully.')
        # Tratamento de exceções - Tabela já existente
        except self.dynamodb_client.exceptions.ResourceInUseException:
            print(f'Table {table_name} already exists.')
        # Tratamento de exceções - Erro ao criar a tabela
        except ClientError as e:
            print(f'Error creating table: {e}')

    # Função para criar o bucket S3 com o nome do bucket
    def create_s3_bucket(self, bucket_name):
        # Criação do bucket com o nome do bucket
        try:
            self.s3_client.create_bucket(Bucket=bucket_name)
            print(f'Bucket {bucket_name} created successfully.')
        # Tratamento de exceções - Bucket já existente
        except self.s3_client.exceptions.BucketAlreadyExists:
            print(f'Bucket {bucket_name} already exists.')
        # Tratamento de exceções - Erro ao criar o bucket
        except ClientError as e:
            print(f'Error creating bucket: {e}')
            
    # Função para criar a função IAM com o nome da função
    def create_iam_role(self, role_name):
        assume_role_policy_document = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "sagemaker.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole"
                }
            ]
        }
        try:
            # Criação da função com o nome da função e a política de confiança
            self.iam_client.create_role(
                RoleName=role_name,
                AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
            )
            # Anexando políticas à função criada para acesso ao S3, DynamoDB e SageMaker
            self.iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
            )
            self.iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonDynamoDBFullAccess'
            )
            self.iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
            )
            self.iam_client.attach_role_policy(
                RoleName=role_name,
                PolicyArn='arn:aws:iam::aws:policy/AWSLambda_FullAccess'
            )
            print(f'Role {role_name} created successfully.')
        # Tratamento de exceções - Função já existente
        except self.iam_client.exceptions.EntityAlreadyExistsException:
            print(f'Role {role_name} already exists.')
        # Tratamento de exceções - Erro ao criar a função
        except ClientError as e:
            print(f'Error creating role: {e}')

# Criar e configurar os recursos AWS
aws_manager = AWSResourceManager(profile_name)
aws_manager.create_dynamodb_table(table_name)
aws_manager.create_s3_bucket(bucket_name)
aws_manager.create_iam_role(role_name)

Table HotelReservations-test3 already exists.
Bucket hotel-reservations-bucket-test3 created successfully.
Role AmazonSageMaker-ExecutionRole already exists.
